In [0]:
%sql
USE pinterest_data;



In [0]:
SELECT * FROM df_pin

category,description,downloaded,follower_count,image_src,index,is_image_or_video,poster_name,save_location,tag_list,title,unique_id
diy-and-crafts,"Make this adorable (and easy) DIY doll crib from an old cardboard box, your little one will love to rock their dolls to sleep in this fun upcycled project!",1,66k,https://i.pinimg.com/originals/62/ee/0d/62ee0d79f92248b2d5819b8bfdd94551.jpg,3020,image,"Cassie May - Little Red Window Crafts, DIY, crafts, tutorials and knitting patterns! Cassie May - Little Red Window Crafts, DIY, crafts, tutorials and knitting patterns! Edit settings Cassie May - Little Red Window Crafts, DIY, crafts, tutorials and knitting patterns!",Local save in /data/diy-and-crafts,"Easy Sewing Projects,Sewing Projects For Beginners,Sewing Hacks,Diy Dolls Crib,Doll Beds,Diy Baby Headbands,Diy Cardboard,Minky Fabric,Sewing Basics",Cardboard DIY Doll Crib,186a847a-6778-48c0-888c-4578550ce7c7
diy-and-crafts,"Make this adorable (and easy) DIY doll crib from an old cardboard box, your little one will love to rock their dolls to sleep in this fun upcycled project!",1,66k,https://i.pinimg.com/originals/62/ee/0d/62ee0d79f92248b2d5819b8bfdd94551.jpg,3020,image,"Cassie May - Little Red Window Crafts, DIY, crafts, tutorials and knitting patterns! Cassie May - Little Red Window Crafts, DIY, crafts, tutorials and knitting patterns! Edit settings Cassie May - Little Red Window Crafts, DIY, crafts, tutorials and knitting patterns!",Local save in /data/diy-and-crafts,"Easy Sewing Projects,Sewing Projects For Beginners,Sewing Hacks,Diy Dolls Crib,Doll Beds,Diy Baby Headbands,Diy Cardboard,Minky Fabric,Sewing Basics",Cardboard DIY Doll Crib,186a847a-6778-48c0-888c-4578550ce7c7
home-decor,"6,636 Likes, 141 Comments - The Cottage Journal (@thecottagejournal) on Instagram: “Can you say color?! 😍😍😍 We are loving the cheery vibes that these aqua blue cabinets are g…",1,394,https://i.pinimg.com/originals/8c/17/a2/8c17a257b70780480bb89c3699363144.jpg,6633,image,Sarah Martin,Local save in /data/home-decor,"Diy Kitchen Cabinets,Kitchen Redo,Home Decor Kitchen,New Kitchen,Home Kitchens,Kitchen Remodeling,Aqua Kitchen,Kitchen Counters,Kitchen Islands",The Cottage Journal on Instagram: “Can you say color?! 😍😍😍 We are loving the cheery vibes that these aqua blue cabinets are giving. If you could paint your cabinets any…”,d136f6bc-840d-44f8-bbad-115eb7e6c51e
home-decor,"6,636 Likes, 141 Comments - The Cottage Journal (@thecottagejournal) on Instagram: “Can you say color?! 😍😍😍 We are loving the cheery vibes that these aqua blue cabinets are g…",1,394,https://i.pinimg.com/originals/8c/17/a2/8c17a257b70780480bb89c3699363144.jpg,6633,image,Sarah Martin,Local save in /data/home-decor,"Diy Kitchen Cabinets,Kitchen Redo,Home Decor Kitchen,New Kitchen,Home Kitchens,Kitchen Remodeling,Aqua Kitchen,Kitchen Counters,Kitchen Islands",The Cottage Journal on Instagram: “Can you say color?! 😍😍😍 We are loving the cheery vibes that these aqua blue cabinets are giving. If you could paint your cabinets any…”,d136f6bc-840d-44f8-bbad-115eb7e6c51e
home-decor,"И хоть у шведов любимые цвета - серый и белый, интерьеры этого конкретного загородного коттеджа в Швеции наполнены теплыми натуральными оттенками. В такой ✌PUFIK. Beautiful Inte…",1,136k,https://i.pinimg.com/originals/3a/7d/8e/3a7d8ecdf8b82f06885dc9763f9ba3d0.png,6447,image,PUFIK Interiors & Inspirations,Local save in /data/home-decor,"Cheap Home Decor,Diy Home Decor,Room Decor,Cozy Cottage,Cozy House,Shabby Cottage,Shabby Chic,Decoration Hall,House Decorations",〚 Warm natural tones and vintage decor: cozy cottage in Sweden 〛◾ Photos ◾ Ideas ◾ Design,d3039535-5767-426a-ba80-cbda6e0e008a
home-decor,"И хоть у шведов любимые цвета - серый и белый, интерьеры этого конкретного загородного коттеджа в Швеции наполнены теплыми натуральными оттенками. В такой ✌PUFIK. Beautiful Inte…",1,136k,https://i.pinimg.com/originals/3a/7d/8e/3a7d8ecdf8b82f06885dc9763f9ba3d0.png,6447,image,PUFIK Interiors & Inspirations,Local save i

In [0]:
SELECT description, COUNT(*) AS repeat_count
FROM df_pin
GROUP BY description
ORDER BY repeat_count DESC
LIMIT 50;


description,repeat_count
No description available Story format,90
No description available,18
"On our Calgary to Vancouver road trip we got to spend time in Banff, Canada. We visited the stunning Lake Moraine as well as Lake Louise and plenty of other of Canada's natural…",5
Amsterdam is one of the top 20 travel destinations for 2020. Click here for the full list. #amsterdam #netherlands #europe #europetravel #bucketlist #travel | once in a lifetime…,4
"From medieval city concept art up to fantasy and medieval castle concept art, our gallery depicts the imagination of extraordinary artists materialized!",4
"8 Followers, 11 Following, 0 Posts - See Instagram photos and videos from Giuseppe Leonardi (@giuse_leonardi)",4
#lovequotes #matchmaker #matchmadeinheaven #loveyourself #respectyourself,4
All I want for Christmas is a beautifully decorated home.,4
"Although in the past, finger tattoos were not chosen by most people. However, as the trend of tattoos continues to change, we have found that in recent years more and more peopl…",4
"Dreaming of traveling world? Saving for such a trip ain't easy, so instead, here's how I created a career out of travel and how I make money while traveling",4


In [0]:
CREATE OR REPLACE TABLE df_pin_clean AS
    SELECT 
        category,
        CASE 
            WHEN description IN ('No description available Story format', 'No description available') 
            THEN NULL
            ELSE description
        END AS description,
        downloaded,
        follower_count,
        image_src,
        `index`,
        is_image_or_video,
        poster_name,
        save_location,
        tag_list,
        title,
        unique_id
    FROM df_pin;

num_affected_rows,num_inserted_rows


In [0]:
CREATE OR REPLACE TABLE df_pin_clean AS
  SELECT *
  FROM df_pin_clean
  WHERE follower_count != 'User Info Error';


num_affected_rows,num_inserted_rows


In [0]:
SELECT image_src, COUNT(*) AS repeat_count
FROM df_pin_clean
GROUP BY image_src
ORDER BY repeat_count DESC
LIMIT 10;

image_src,repeat_count
Image src error.,14
https://i.pinimg.com/videos/thumbnails/originals/57/9d/52/579d52242ae5609978c754d393e44e81.0000001.jpg,5
https://i.pinimg.com/originals/a5/83/25/a58325576095f81bc0013f172d49fa76.jpg,4
https://i.pinimg.com/originals/c6/27/78/c62778ea755247f9aee39a63f167db59.jpg,4
https://i.pinimg.com/originals/1d/f2/45/1df2455b3eb19d710e166765e8fe05c9.png,4
https://i.pinimg.com/videos/thumbnails/originals/15/e5/50/15e550d4281c8fd83bf73076ec1c5bab.0000001.jpg,4
https://i.pinimg.com/originals/8e/df/30/8edf3015c8e8a4ea3d1ba4b2c66dc36f.jpg,4
https://i.pinimg.com/originals/e9/6d/4d/e96d4dc5f4175a1633294a0d016387df.jpg,4
https://i.pinimg.com/originals/c6/64/ee/c664ee71524fb5a6e7b7b49233f93b43.png,4
https://i.pinimg.com/originals/47/53/20/475320d752c260c4372737253b1f81ca.jpg,4


In [0]:
CREATE OR REPLACE TABLE df_pin_clean AS
    SELECT 
        category,
        description,
        downloaded,
        follower_count,
        CASE 
            WHEN image_src IN ('Image src error.') 
            THEN NULL
            ELSE image_src
        END AS image_src,
        `index`,
        is_image_or_video,
        poster_name,
        save_location,
        tag_list,
        title,
        unique_id
    FROM df_pin_clean;

num_affected_rows,num_inserted_rows


In [0]:
SELECT tag_list, COUNT(*) AS repeat_count
FROM df_pin_filtered_3
GROUP BY tag_list
ORDER BY repeat_count DESC
LIMIT 10;

tag_list,repeat_count
"N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e",9
"Beautiful Places To Travel,Cool Places To Visit,Beautiful Roads,Beautiful Scenery,Wonderful Places,Lac Moraine,Moraine Lake,Lac Louise,Natur Wallpaper",5
"Fantasy City,Fantasy Places,Fantasy World,Dark Fantasy,Fantasy Village,Fantasy Castle,Medieval Fantasy,Fantasy Art Landscapes,Fantasy Landscape",4
"Cool Places To Visit,Places To Travel,Places To Go,Bucket List Destinations,Travel Destinations,Samana,Destination Voyage,Amsterdam Canals,Amsterdam Netherlands",4
"Fun Places To Go,Beautiful Places To Travel,Top Travel Destinations,I Want To Travel,Modern City,Athens Greece,Ultimate Travel,Dream Vacations,Travel Around The World",4
"Arizona Road Trip,Sedona Arizona,Arizona Travel,Arizona Usa,Prescott Arizona,Sedona Things To Do,Places To Travel,Places To See,Travel Destinations",4
"Best Money Saving Tips,Money Tips,Saving Money,Budgeting Finances,Budgeting Tips,Financial Tips,Financial Planning,Financial Literacy,Look Here",4
"Motivation Positive,Fitness Motivation Quotes,Positive Quotes,Health Fitness Quotes,Motivation To Work Out,Fitness Motivation Wallpaper,Sport Motivation,Motivation For Exercise,Student Motivation Quotes",4
"Illustration Simple,Pay Off Mortgage Early,Paying Off Credit Cards,Mortgage Tips,Refinance Mortgage,Mortgage Calculator,Mortgage Estimator,Mortgage Humor,Mortgage Payment",4
"Wise Quotes,Quotable Quotes,Words Quotes,Wise Words,Quotes To Live By,Great Quotes,Motivational Quotes,Inspirational Quotes,Funny Quotes",4


In [0]:
CREATE OR REPLACE TABLE df_pin_clean AS
  SELECT 
      category,
      description,
      downloaded,
      follower_count,
      image_src,
      `index`,
      is_image_or_video,
      poster_name,
      save_location,
      CASE 
        WHEN tag_list IN ('N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e') 
        THEN NULL
        ELSE tag_list
      END AS tag_list,
      title,
      unique_id
  FROM df_pin_clean;

num_affected_rows,num_inserted_rows


In [0]:

CREATE OR REPLACE TABLE df_pin_clean AS
SELECT
    category,
    description,
    downloaded,
    CASE
      WHEN follower_count LIKE '%k' THEN CAST(CAST(SUBSTRING(follower_count, 1, LENGTH(follower_count) - 1) AS DECIMAL) * 1000 AS INT) -- removes 'k' from value, multiplies by 1000 and converts to INT
      WHEN follower_count LIKE '%M' THEN CAST(CAST(SUBSTRING(follower_count, 1, LENGTH(follower_count) - 1) AS DECIMAL) * 1000000 AS INT) -- removes 'm' from value, multiplies by 1000000 and converts to INT
      ELSE CAST(follower_count AS INT)
    END AS follower_count,
    image_src,
    `index`,
    is_image_or_video,
    poster_name,
    save_location,
    tag_list,
    title,
    unique_id
FROM df_pin_clean;


num_affected_rows,num_inserted_rows


In [0]:
CREATE OR REPLACE TABLE df_pin_clean AS
SELECT
    category,
    description,
    downloaded,
    follower_count,
    image_src,
    `index`,
    is_image_or_video,
    poster_name,
    TRIM(SUBSTRING(save_location, 16)) AS save_location,  -- removes 'Local save in' prefix
    tag_list,
    title,
    unique_id
FROM df_pin_clean;

num_affected_rows,num_inserted_rows


In [0]:
CREATE OR REPLACE TABLE df_pin_clean AS
SELECT
    category,
    description,
    downloaded,
    follower_count,
    image_src,
    `index` AS ind,  -- Renaming column 'index' to 'ind'
    is_image_or_video,
    poster_name,
    save_location,
    tag_list,
    title,
    unique_id
FROM df_pin_clean;

num_affected_rows,num_inserted_rows


In [0]:
CREATE OR REPLACE TABLE df_pin_clean AS
SELECT
    ind,
    unique_id,
    title,
    description,
    follower_count,
    poster_name,
    tag_list,
    is_image_or_video,
    image_src,
    save_location,
    category
FROM df_pin_clean;

num_affected_rows,num_inserted_rows


In [0]:
CREATE OR REPLACE TABLE df_pin AS
SELECT * FROM df_pin_clean;



num_affected_rows,num_inserted_rows


In [0]:
SELECT * FROM df_pin LIMIT 10;

ind,unique_id,title,description,follower_count,poster_name,tag_list,is_image_or_video,image_src,save_location,category
6439,cc97d817-dc51-439b-b745-1c82ea513f37,IKEA hacks: 10 ways to glam-hack IKEA furniture - Inredningsvis,(Innehåller Ellos Jotex Newport Soffadirekt annonslänkar) I’m going to round off this budget week with some smoking hot hacks from people that have realised that Ikea is so much…,9000,Inredningsvis | Inredning inspiration Interior inspiration,"Living Room Inspiration,Home Decor Inspiration,Decor Ideas,Decor Diy,Creative Inspiration,Interior Design Living Room,Living Room Designs,Interior Livingroom,Living Room Styles",image,https://i.pinimg.com/originals/aa/d5/19/aad519780ee2dc281df65a1a48796687.jpg,data/home-decor,home-decor
8304,5b6d0913-25e4-43ab-839d-85d5516f78a4,The #1 Reason You’re Not His Priority Anymore - Matthew Coast,#lovequotes #matchmaker #matchmadeinheaven #loveyourself #respectyourself,51000,Commitment Connection,"Wise Quotes,Quotable Quotes,Words Quotes,Wise Words,Quotes To Live By,Great Quotes,Motivational Quotes,Inspirational Quotes,Funny Quotes",image,https://i.pinimg.com/originals/c6/64/ee/c664ee71524fb5a6e7b7b49233f93b43.png,data/quotes,quotes
1313,44662045-e891-4821-8a19-ebe7eedd371a,Liquid Lash Extensions Mascara,"Instantly create the look of lash extensions with this award-winning, best-selling mascara that won't clump, flake or smudge. Available in 3 shades!",43000,Thrive Causemetics,null,video,https://i.pinimg.com/videos/thumbnails/originals/69/84/e2/6984e20f3e262098fa9c0614c3453254.0000001.jpg,data/beauty,beauty
784,a781c693-0288-4f4c-930f-7b9c198168b3,Spooky Tree Circle Art,Arty Crafty Kids - Spooky Tree Circle Art for Halloween. Explore cool and warm shades with this simple template design. Perfect Art Project for kids exploring colour mixing and…,252000,Arty Crafty Kids,"Halloween Kunst,Halloween Art Projects,Fall Art Projects,Halloween Activities For Kids,Halloween Painting,Projects For Kids,Simple Art Projects,Art Club Projects,Art Activities",image,https://i.pinimg.com/originals/da/ce/6e/dace6e1df60da6eb7ce689403bff05ef.jpg,data/art,art
5494,8fb2af68-543b-4639-8119-de33d28706ed,Dave Ramsey's 7 Baby Steps: What Are They And Will They Work For You,"If you love budgeting, make sure to give Dave Ramsey's 7 Baby Steps a try. Follow these steps to begin your debt snowball, build an emergency fund, invest and reach riches. I ca…",26000,"Living Low Key | Save Money, Make Money, & Frugal Living","Financial Peace,Financial Tips,Saving Money Quotes,Total Money Makeover,Budgeting Finances,Money Management,Wealth Management,Personal Finance,Making Ideas",image,https://i.pinimg.com/originals/1e/9d/90/1e9d906e4e150e3b95187f3b76ea7c71.png,data/finance,finance
2923,52fa3af5-24a4-4ccb-8f17-9c3eb12327ee,UFO Paper Plate Craft,A fun space activity for kids. Preshoolers and kindergartners will love making their own alien spacecraft!,192000,The Crafting Chicks,"Paper Plate Crafts For Kids,Fun Crafts For Kids,Summer Crafts,Toddler Crafts,Art For Kids,Outer Space Crafts For Kids,Kid Crafts,Space Kids,Back To School Crafts For Kids",image,https://i.pinimg.com/originals/6f/e8/aa/6fe8aa405513c6d2f77b5f47d17cdce8.jpg,data/diy-and-crafts,diy-and-crafts
6063,60693727-4927-4bd6-a8c5-096a392d63e6,41 Gorgeous Fall Decor Ideas For Your Home - Chaylor & Mads,"Beautiful and easy ways to update every room in your home with fall decor. Plus, my favorite finds in fall decor for 2020!",92000,"Kristen | Lifestyle, Mom Tips & Teacher Stuff Blog","Fall Home Decor,Autumn Home,Fall Decor Outdoor,Front Porch Fall Decor,Home Decor Ideas,Porch Ideas For Fall,Fall Outdoor Decorating,Decorating Ideas For Fall,Fall Front Doors",image,https://i.pinimg.com/originals/e5/ae/dc/e5aedc14ce557e3a69f672e0f8c88f6e.png,data/home-decor,home-decor
6603,e6d4755d-d139-4d92-8642-a0dcfd32bcab,Build Your Own DIY Kitchen Island | Tutorial & Step-by-Step Building Plans,How to Build a DIY Furniture Style Kitchen Island & Free Plans,44000,Nina H